In [1]:
# Import dependencies
from textblob import TextBlob
import pandas as pd
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import time
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install gensim
import gensim
from gensim.parsing.preprocessing import remove_stopwords 
import torch
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/memme11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [504]:
# Set Column width to max
pd.set_option('display.max_colwidth', None)

In [2]:
# Read in sample csv (Twitter analysis)
btc_twitter = pd.read_csv('Bitcoin_tweets.csv')
btc_twitter

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423106,🐥🍀,NaN,#banditgang big win pls 🙏🍀,2018-01-14 07:33:25,17.0,1324.0,1870.0,False,2021-07-02 09:06:24,@earnbnb $bsc $btc #BNB #Bitcoin #BSC #earn...,"['BNB', 'Bitcoin', 'BSC', 'earnBNB', 'fairlaun...",Twitter for iPhone,False
423107,Juliana,NaN,Pejuang rizki allah 🙏\nSemangat ibadah,2021-06-28 05:27:54,11.0,212.0,1058.0,False,2021-07-02 09:06:21,@GiveAwayHost Bismillah\nWish me luck \n#Bitcoin,['Bitcoin'],Twitter for Android,False
423108,Der Zeitgeist,"Kiel, Schleswig-Holstein","Was ihr den Geist der Zeiten heißt, Das ist im...",2013-02-11 15:28:36,1929.0,1803.0,5252.0,False,2021-07-02 09:06:21,What do we learn from this? Don't mess with th...,"['ElSalvador', 'BTC', 'Bitcoin', 'mining', 'IM...",Twitter Web App,False
423109,icon0.com,NaN,"#icon0 Free images for personal , commercial a...",2010-04-11 06:22:55,599.0,15.0,3324.0,False,2021-07-02 09:06:20,Download Bitcoin icon sign design\n Free image...,"['bitcoin', 'btc', 'cryptocurrency', 'crypto',...",Twitter Web App,False


In [3]:
# Clean 
# Drop columns
btc_twitter = btc_twitter.drop(columns=['user_name', 'user_location', 'user_description', 'user_created', 'user_verified', 'source'])

# Clean date and text columns using regex
btc_twitter['date'] = btc_twitter['date'].str.extract('(2021-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')

btc_twitter['cleaned_text'] = btc_twitter['text']
clean_text = '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)'
btc_twitter['cleaned_text'] = btc_twitter['cleaned_text'].str.replace(clean_text, " ", regex=True)
btc_twitter['cleaned_text'] = btc_twitter['cleaned_text'].str.lower()

# Re-order columns 
columns = ['text', 'cleaned_text', 'user_followers', 'user_friends', 'user_favourites', 'hashtags', 'is_retweet', 'date']
btc_twitter = btc_twitter[columns]

# Convert date dtype to datetime
btc_twitter['date'] = pd.to_datetime(btc_twitter['date'])

# Keep only tweets with user_followers > 5000
btc_twitter = btc_twitter.loc[btc_twitter['user_followers']>5000]

btc_twitter

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet,date
0,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted by nyse after b...,8534.0,7605,4838,['bitcoin'],False,2021-02-10 23:59:04
1,"😎 Today, that's this #Thursday, we will do a ""...",today that s this thursday we will do a ...,6769.0,1532,25483,"['Thursday', 'Btc', 'wallet', 'security']",False,2021-02-10 23:58:48
9,.@Tesla’s #bitcoin investment is revolutionary...,s bitcoin investment is revolutionary for ...,8534.0,7605,4838,"['bitcoin', 'crypto']",False,2021-02-10 23:52:08
12,Join our first virtual crypto meetup of 2021 -...,join our first virtual crypto meetup of 2021 ...,5097.0,791,52,NaN,False,2021-02-10 23:50:59
16,@naval #BTC is unconfiscatable \n\nAll roads l...,btc is unconfiscatable all roads lead to ...,72542.0,519,19763,"['BTC', 'Bitcoin']",False,2021-02-10 23:48:37
...,...,...,...,...,...,...,...,...
423045,What Problems Does Bitcoin Solve? https://t.co...,what problems does bitcoin solve cryptocur...,14982.0,5832.0,16169.0,"['cryptocurrency', 'crypto', 'digital', 'bitco...",False,2021-07-02 09:10:02
423046,"Live #Bitcoin Price: $33,377\n\nThat's a decre...",live bitcoin price 33 377 that s a decreas...,39946.0,7518.0,970.0,"['Bitcoin', 'Bitcoin', 'Binance', 'Crypto', 'D...",False,2021-07-02 09:10:02
423065,Bitcoin bounces at key range amid hope BTC pri...,bitcoin bounces at key range amid hope btc pri...,9276.0,175.0,0.0,"['news', 'investing', 'bitcoin']",False,2021-07-02 09:09:15
423092,I heard that the bull market is over.\n\nThe d...,i heard that the bull market is over the dar...,8702.0,159.0,64.0,['Bitcoin'],False,2021-07-02 09:07:04


In [4]:
# Check data types
btc_twitter.dtypes

text                       object
cleaned_text               object
user_followers            float64
user_friends               object
user_favourites            object
hashtags                   object
is_retweet                 object
date               datetime64[ns]
dtype: object

In [5]:
# Reformat
sorted_btc = btc_twitter.set_index('date').sort_index(ascending=False)
sorted_btc.dropna(inplace=True)
sorted_btc.drop_duplicates(inplace=True)
sorted_btc

,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet
date,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroa...,thailand boutiquehotels for sale willroad...,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse'...",False
2021-07-05 23:26:36,LUNALAND ($LLN) will burn coin supply until we...,lunaland lln will burn coin supply until we...,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'in...",False
2021-07-05 23:25:27,#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD ...,lightningnetwork observed 22 365 nodes ...,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitco...",False
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is...",barclay s just wants to keep you safe this is...,20668.0,451.0,0.0,['bitcoin'],False
2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.c...,why cardano is just teasing us ada alt...,9276.0,175.0,0.0,"['altcoin', 'news', 'bitcoin']",False
...,...,...,...,...,...,...,...
2021-02-05 11:00:24,"5⃣ #Blockchain 50 2021 by @DelRayMan, @Forbe...",5 blockchain 50 2021 by cry...,16813.0,818,1201,"['Blockchain', 'cryptocurrency', 'bitcoin', 'c...",False
2021-02-05 11:00:24,4⃣ 🎙️ Bloomberg LP #CryptoOutlook 2021 with @...,4 bloomberg lp cryptooutlook 2021 with ...,16813.0,818,1201,"['CryptoOutlook', 'cryptocurrency', 'bitcoin',...",False
2021-02-05 11:00:22,2⃣ Debunking 9 #Bitcoin Myths by @Patrick_Lo...,2 debunking 9 bitcoin myths by lowry ...,16813.0,818,1201,"['Bitcoin', 'cryptocurrency', 'bitcoin', 'cryp...",False


In [6]:
# Output to csv
output_file = 'Cleaned_Bitcoin_tweets.csv'
sorted_btc.to_csv(output_file, index='inplace')